<a href="https://colab.research.google.com/github/cauancalmeida/Vendas_Shop_Now/blob/main/Analise_Shop_Now.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analise de vendas Shop Now

##Carregar dados

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [89]:
df=pd.read_csv('https://raw.githubusercontent.com/cauancalmeida/Vendas_Shop_Now/refs/heads/main/vendas_shopnow.csv')

In [90]:
df.head()

,order_id,customer_id,order_date,product_category,quantity,price,payment_method
0,O1000,C29,2023-02-24,NaN,4,262.52,PayPal
1,O1001,C168,2023-03-11,Esporte,5,416.70,PayPal
2,O1002,C161,2023-10-30,Roupas,3,173.62,Pix
3,O1003,C175,2023-11-06,Esporte,5,449.85,Pix
4,O1004,C153,2023-11-26,Livros,5,206.82,Pix


##Informações das colunas

order_id – ID único do pedido

customer_id – ID único do cliente

order_date – Data do pedido

product_category – Categoria do produto

quantity – Quantidade comprada

price – Preço unitário do produto (em R$)

payment_method – Método de pagamento utilizado

##Tratando os dados

In [91]:
df.info()#coletando informações gerais do df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          500 non-null    object 
 1   customer_id       500 non-null    object 
 2   order_date        500 non-null    object 
 3   product_category  495 non-null    object 
 4   quantity          500 non-null    int64  
 5   price             500 non-null    float64
 6   payment_method    495 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 27.5+ KB


In [92]:
df.shape #verificando o tamnho do df

(500, 7)

In [93]:
df.describe() #verificação de informações numericas

,quantity,price
count,500.000000,500.000000
mean,2.970000,251.205440
std,1.425891,147.536195
min,1.000000,-406.980000
25%,2.000000,121.312500
50%,3.000000,247.045000
75%,4.000000,378.272500
max,5.000000,499.860000


In [94]:
df.describe(include='object')#verificação de informações com textos

,order_id,customer_id,order_date,product_category,payment_method
count,500,500,500,495,495
unique,500,185,270,6,4
top,O1499,C140,2023-03-11,Livros,Boleto
freq,1,6,7,91,133


In [95]:
df['product_category'].value_counts() #verificando as categorias de produto

,count
product_category,
Livros,91
Esporte,90
Roupas,82
Eletrônicos,80
Beleza,77
Casa e Jardim,75


In [96]:
df['payment_method'].value_counts() #verificando os tipos de pagamento

,count
payment_method,
Boleto,133
Pix,126
PayPal,121
Cartão de Crédito,115


###Limpando os dados nulos

In [97]:
df.replace('',np.nan, inplace=True) #transformando qualquer valor vazio em nulo para ser tratado junto

In [98]:
df.isna().sum()

,0
order_id,0
customer_id,0
order_date,0
product_category,5
quantity,0
price,0
payment_method,5


In [99]:
clientes_na=df[df['product_category'].isna()] #pega o Id dos clientes que tem a categoria nula

In [100]:
#Completa a categoria de produto com o historico de cada cliente
for i in clientes_na['customer_id'].unique():
  client = df[df['customer_id']==i] #mapeia cada cliente que tem a categoria nula
  categoria_nula = client[client["product_category"].isna()] #pega a categoria nula de cada cliente
  categorias_existentes = client[client["product_category"].notna()] #pega as cateorias nã nulas
  categorias_frequentes = categorias_existentes["product_category"].value_counts() #conta a frequencia de cada categoria
  categoria_mais_frequente = categorias_frequentes.idxmax() #pega a categoria mais frequente
  df.loc[categoria_nula.index, "product_category"] = categoria_mais_frequente #preenche a categoria nula com a categoria mais frequente



In [101]:
df['payment_method'].fillna('Outros', inplace=True) #preenche os valores nulos com Outros

/tmp/ipython-input-1644425482.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['payment_method'].fillna('Outros', inplace=True) #preenche os valores nulos com Outros


In [102]:
df.isna().sum() #verificando se ainda tem algum campo nulo

,0
order_id,0
customer_id,0
order_date,0
product_category,0
quantity,0
price,0
payment_method,0


In [103]:
df.duplicated().sum() #verificando se existem dados duplicados

np.int64(0)

###Verificando dados numericos

In [104]:
preco_negativo = df[df['price']<0].index #verificando se existem preços negativos

In [105]:
media_preco = df.groupby('product_category')['price'].mean() #verificando a média de preço por categoria

In [106]:
df.loc[preco_negativo,'price']= df.loc[preco_negativo, 'product_category'].map(media_preco) #preenchendo os preços negativos com a média de preço por categoria

In [107]:
df[df['price']<0]

,order_id,customer_id,order_date,product_category,quantity,price,payment_method
